# Pretraining tail network

In [1]:
from src.dlc_practical_prologue import generate_pair_sets
from src.utils import load_class_data, load_target_data, load_all_data, print_param_count
from src.models import *
from src.trainer import Trainer
import torch
import matplotlib.pyplot as plt

In [2]:
# Load data with class and larget than as targets
dl_train_all, dl_val_all, dl_test_all = load_all_data()

# Load data with class as target
dl_train_class, dl_val_class, dl_test_class = load_class_data()

In [3]:
print("--- Pre-training tail model ---\n")

tail = TailLinear(label_encoded=True)
trainer = Trainer(nb_epochs=5)
trainer.fit(tail, dl_train_all, dl_val_all)

print("\n--- Training full model ---\n")

tail.label_encoded = False
tail.requires_grad_=False
le_net = LeNet()
siamese = Siamese(le_net, tail)
trainer = Trainer(nb_epochs=20)
trainer.fit(siamese, dl_train_all, dl_val_all)

print("\n--- Testing model ---\n")
trainer.test(le_net, dl_test_class, return_acc=False)
trainer.test(siamese, dl_test_all, return_acc=False)


--- Pre-training tail model ---

# Epoch 1/5:	 loss=0.65	 loss_val=0.6	 acc_val=62.5
# Epoch 2/5:	 loss=0.51	 loss_val=0.4	 acc_val=87.05
# Epoch 3/5:	 loss=0.29	 loss_val=0.18	 acc_val=99.11
# Epoch 4/5:	 loss=0.12	 loss_val=0.08	 acc_val=100.0
# Epoch 5/5:	 loss=0.06	 loss_val=0.05	 acc_val=100.0

--- Training full model ---

# Epoch 1/20:	 loss=2.46	 loss_val=0.4	 acc_val=81.25
# Epoch 2/20:	 loss=1.25	 loss_val=0.42	 acc_val=82.59
# Epoch 3/20:	 loss=0.9	 loss_val=0.32	 acc_val=84.38
# Epoch 4/20:	 loss=0.67	 loss_val=0.32	 acc_val=82.14
# Epoch 5/20:	 loss=0.57	 loss_val=0.33	 acc_val=84.38
# Epoch 6/20:	 loss=0.5	 loss_val=0.27	 acc_val=87.5
# Epoch 7/20:	 loss=0.42	 loss_val=0.26	 acc_val=90.18
# Epoch 8/20:	 loss=0.39	 loss_val=0.28	 acc_val=85.71
# Epoch 9/20:	 loss=0.36	 loss_val=0.27	 acc_val=88.84
# Epoch 10/20:	 loss=0.31	 loss_val=0.23	 acc_val=89.73
# Epoch 11/20:	 loss=0.26	 loss_val=0.25	 acc_val=90.18
# Epoch 12/20:	 loss=0.3	 loss_val=0.23	 acc_val=90.18
# Epoch 13/2

In [4]:
print_param_count(siamese)

Total number of parameters:     83300
Number of trainable parameters: 83300


In [8]:
# Do the same, but 10 times and with verbose=False
results = []
for i in range(10):
    print(f"Run {i+1}/10")

    # Load data with class and larget than as targets
    dl_train_all, dl_val_all, dl_test_all = load_all_data()

    # Load data with class as target
    dl_train_class, dl_val_class, dl_test_class = load_class_data()

    # Training tail model
    tail = TailLinear(label_encoded=True)
    trainer = Trainer(nb_epochs=5)
    trainer.fit(tail, dl_train_all, dl_val_all, verbose=False)

    # Training full model (but without gradient for last part)
    tail.label_encoded = False
    tail.requires_grad_=False
    le_net = LeNet()
    siamese = Siamese(le_net, tail)
    trainer = Trainer(nb_epochs=20)
    trainer.fit(siamese, dl_train_all, dl_val_all, verbose=False)

    # Testing
    acc = trainer.test(siamese, dl_test_all, test_verbose=True, return_acc=True)
    results.extend([acc])

Run 1/10
loss_test=0.31	 acc_test=90.62
Run 2/10
loss_test=0.26	 acc_test=92.58
Run 3/10
loss_test=0.44	 acc_test=89.16
Run 4/10
loss_test=0.4	 acc_test=92.29
Run 5/10
loss_test=0.37	 acc_test=90.23
Run 6/10
loss_test=0.28	 acc_test=93.46
Run 7/10
loss_test=0.34	 acc_test=91.89
Run 8/10
loss_test=0.27	 acc_test=92.38
Run 9/10
loss_test=0.39	 acc_test=90.62
Run 10/10
loss_test=0.44	 acc_test=90.33


In [9]:
# Print results
results = torch.Tensor(results)
print("Average accuracy:   {:.2f}".format(results.mean()))
print("Standard deviation: {:.2f}".format(results.std()))

Average accuracy:   91.36
Standard deviation: 1.35
